<a href="https://colab.research.google.com/github/gabrielterceiro/ReadPDF/blob/main/sale_extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#install pdfplumber package to extract the text from pdf
!pip install pdfplumber

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
#imports
import pdfplumber
import re
from collections import namedtuple
import pandas as pd

In [3]:
#create the named tuple pattern
Sale = namedtuple('Sale', 'dt_sale cd_prod nm_prod qt_prod vl_prod')

In [4]:
#using pdfplumber to open the semi structured pdf file and add the text from all pages to a variable
with pdfplumber.open('/content/DadosAnalise.pdf') as pdf:
    text = ''
    for page in pdf.pages:
        text += page.extract_text() 

In [5]:
#create regular expressions to find the sale dates and sale details
new_dt_re = re.compile(r'^([D][at]{3}) (.*)') 
new_sale_re = re.compile(r'\d+ [\w\s]+ (\d+) (.*) ([\d,]+\d{2}) [0] ([\d,]+\d{2})')

In [6]:
#initialize an empty list to append later
lst_sale = []

#run through every line of the text
for line in text.split('\n'):
    #if the line contains a date, stores the date in a variable
    if new_dt_re.match(line):
        dt_string, dt_sale = line.split()

    #if the line contains a sale, stores the important information in their respective variables
    line = new_sale_re.search(line)
    if line:
        cd_prod = line.group(1)
        nm_prod = line.group(2)
        qt_prod = line.group(3)
        vl_prod = line.group(4)
        #append into the list the named tuple with the sale data
        lst_sale.append(Sale(dt_sale, cd_prod, nm_prod, qt_prod, vl_prod))

In [7]:
#stores the data into a dataframe
df = pd.DataFrame(lst_sale)

In [8]:
df.head(10)

,dt_sale,cd_prod,nm_prod,qt_prod,vl_prod
0,02/05/2022,2,PICOLE CREME,"2,00","5,00"
1,02/05/2022,12,POTE ESPECIAL 2L,"2,00","35,00"
2,02/05/2022,55,KID CASCAO C/ 10 UN,"1,00","7,00"
3,02/05/2022,2,PICOLE CREME,"2,00","5,00"
4,02/05/2022,6,PICOLE MAIS,"4,00","12,00"
5,02/05/2022,9,"POTE GRAN NEVADO 1,5L","1,00","15,00"
6,02/05/2022,12,POTE ESPECIAL 2L,"1,00","17,50"
7,02/05/2022,12,POTE ESPECIAL 2L,"1,00","17,50"
8,02/05/2022,55,KID CASCAO C/ 10 UN,"1,00","7,00"
9,02/05/2022,18,BEST CUP 750ML,"2,00","26,00"
